# Overview
**Tutorial**: recbole-using-all-items-for-prediction

**Author**: [astrung](https://github.com/astrung)

**Original link**: [notebook](https://www.kaggle.com/code/astrung/recbole-using-all-items-for-prediction/notebook)

**Edit**: 
I have create new notebooks for applying our customize function for using all items as input for recommendation:
* Using only interactions: https://www.kaggle.com/astrung/sequential-model-fixed-missing-last-item
* Using interactions with item features: https://www.kaggle.com/code/astrung/lstm-model-with-item-infor-fix-missing-last-item

- - -

In my previous [notebook](https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial) about sequential model with Recbole, someone asked me about the mechanism of test data when using `full_sort_topk` for prediction submitted recommendation in this [comment](https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial/comments#1723707) and this [comment](https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial/comments#1723707), and they have a doubt about whether we are using all items for getting final recommendation. Most of people has 2 questions about using `full_sort_topk` with test data:
1. Do items in test data are used as input features for getting recommendation ?
2. If test data is necessary for getting recommendation in Recbole API, how can we get recommendation without splitting into train/test data?

In this notebook i will answer all questions:
1. Yes. In sequential models, items in test data is used as input features, but not last items. As a example, if user X have 3 items in test data(A, B, C) and 5 items in train data(a,b,c,d,e), test data will generate 3 sample rows for evaluating performance on user X:
* Row 1: Input features: `a,b,c,d,e,0,0`. Output features: `A`. `0` is a pad item
* Row 2: Input features: `a,b,c,d,e,A,0`. Output features: `B`.
* Row 3: Input features: `a,b,c,d,e,A,B`. Output features: `C`.

In my previous notebook, i use last row result as recommendation, **so we still using nearly all of items as input for recommendation, except last item(item C)**. Our recommendation in previous notebooks may be not perfect, but it is simple as a tutorial for anyone want to start.

**Note: This mechanism is only for sequential model in recbole. For other types of model, it isn't correct - it won't use items in test data for getting recommendation. If you have requests for explaining for other model, please upvote and comment. I will explain it in other notebook**

In first session of this notebook, i will dig into test data to prove this conclusion.

2. Yes, we can get recommendation by using all of items as input features, without splitting train/test. In order to do this, you need to modify recbole code:
* Fist, you copy last row in dataset(input features have all items, except last one), then add last item into input features.
* Then you predict directly from model api, without using [full_sort_score or full_sort_topk](https://recbole.io/docs/user_guide/usage/case_study.html)

In second session of this notebook, i will show you how to do that.

Ok, let start

# I. How test items are used in test data.

For each item in test data, it will be generated as a sample row. As a example, if user X have 3 items in test data(A, B, C) and 5 items in train data(a,b,c,d,e), test data will generate 3 sample rows for evaluating performance on user X:
* Row 1: Input features: `a,b,c,d,e,0,0`. Output label: `A`. `0` is a pad item
* Row 2: Input features: `a,b,c,d,e,A,0`. Output label: `B`.
* Row 3: Input features: `a,b,c,d,e,A,B`. Output label: `C`.

For proving it, we will create a dataset, then extract input features and label in test data.

### 1. Let create test data in recbole

In [1]:
!pip install recbole

     |████████████████████████████████| 2.0 MB 537 kB/s            
     |████████████████████████████████| 27.4 MB 99 kB/s              
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: colorlog
    Found existing installation: colorlog 6.6.0
    Uninstalling colorlog-6.6.0:
      Successfully uninstalled colorlog-6.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.6.0 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.5.1 which is incompatible.
featuretools 1.6.0 requires numpy>=1.21.0, but you have numpy 1.20.3 which is incompatible.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is inc

In [2]:
import pandas as pd
df = pd.read_csv(r"/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv", 
                 dtype={'article_id': 'str'})
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [3]:
import numpy as np
df['t_dat'] = pd.to_datetime(df['t_dat'], format="%Y-%m-%d")
df['timestamp'] = df.t_dat.values.astype(np.int64) // 10 ** 9
df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,timestamp
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2,1537401600
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2,1537401600
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2,1537401600
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2,1537401600
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2,1537401600


In [4]:
temp = df[df['timestamp'] > 1585620000][['customer_id', 'article_id', 'timestamp']].rename(
    columns={'customer_id': 'user_id:token', 'article_id': 'item_id:token', 'timestamp': 'timestamp:float'})
temp

,user_id:token,item_id:token,timestamp:float
23934157,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0727808001,1585699200
23934158,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0727808007,1585699200
23934159,000563485cbb7850b0a93c6606f89c5b961c6647d1bd48...,0567532015,1585699200
23934160,000563485cbb7850b0a93c6606f89c5b961c6647d1bd48...,0706104009,1585699200
23934161,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,0783504004,1585699200
...,...,...,...
31788319,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,1600732800
31788320,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,1600732800
31788321,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,1600732800
31788322,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,1600732800


Create data file in recbole format

In [5]:
!mkdir /kaggle/working/recbox_data
temp.to_csv('/kaggle/working/recbox_data/recbox_data.inter', index=False, sep='\t')

In [6]:
import gc
del temp
gc.collect()

21

In [7]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

In [8]:
parameter_dict = {
    'data_path': '/kaggle/working',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 2,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='GRU4Rec', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

Now let start spliting train data and test data in recbole

In [9]:
dataset = create_dataset(config)
logger.info(dataset)

recbox_data
The number of users: 15459
Average actions of users: 59.21956268598784
The number of items: 7330
Average actions of items: 124.9032610178742
The number of inters: 915416
The sparsity of the dataset: 99.19214553975321%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [10]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

[Training]: train_batch_size = [2048] negative sampling: [None]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]


### 2. Let extract sample rows from test data.

We will check items of user `0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef`. 

We except that last items of this user will be used as label in test data


In [11]:
last_item_ids = df[df.customer_id == '0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef'
                  ].tail(10).article_id.values
df[df.customer_id == '0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef'].tail(10)

,t_dat,customer_id,article_id,price,sales_channel_id,timestamp
29315229,2020-07-22,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0842727001,0.027102,2,1595376000
29315230,2020-07-22,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0850241005,0.020322,2,1595376000
29315231,2020-07-22,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0871517013,0.020322,2,1595376000
30421003,2020-08-17,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0903062001,0.040881,2,1597622400
30421004,2020-08-17,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0861478003,0.024525,2,1597622400
30421005,2020-08-17,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0857778011,0.032695,2,1597622400
30421006,2020-08-17,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0698286004,0.024525,2,1597622400
30421007,2020-08-17,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0698286004,0.024525,2,1597622400
30421008,2020-08-17,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0861478002,0.016339,2,1597622400
30421009,2020-08-17,0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8...,0901955001,0.024525,2,1597622400


In [12]:
last_item_ids

array(['0842727001', '0850241005', '0871517013', '0903062001',
       '0861478003', '0857778011', '0698286004', '0698286004',
       '0861478002', '0901955001'], dtype=object)

Recbole use an internal ids for identify user_id and item_id, so let convert this user_id and his items into internal ids.
* customer_id: `0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef` -> internal user id: 2
* last bought item_id: [..., '0698286004', '0861478002', '0901955001'] -> internal item id: [..., 3237, 4377, 4559]

In [13]:
test_data.dataset.token2id(test_data.dataset.uid_field, 
                           '0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef')

2

In [14]:
print(dataset.token2id(dataset.iid_field, last_item_ids))

[ 797 4975 6339 6070 4054 6745 3237 3237 4377 4559]


**Now let extract input features and labels in our test data.
My extracted code is copy from [this source](https://recbole.io/docs/_modules/recbole/utils/case_study.html#full_sort_scores)**

In [15]:
input_features = test_data.dataset[np.isin(test_data.dataset[test_data.dataset.uid_field].numpy(), [2])]
input_features

The batch_size of interaction: 5
    user_id, torch.Size([5]), cpu, torch.int64
    item_id, torch.Size([5]), cpu, torch.int64
    timestamp, torch.Size([5]), cpu, torch.float32
    item_length, torch.Size([5]), cpu, torch.int64
    item_id_list, torch.Size([5, 50]), cpu, torch.int64
    timestamp_list, torch.Size([5, 50]), cpu, torch.float32


* **item_id in above interaction is used as label item**
* **item_id_list in above interaction is used as feature items**

Let check it

In [16]:
print("test label: " + str(input_features['item_id']))
print("last 10 items from origin dataset: " + str(dataset.token2id(dataset.iid_field, last_item_ids)))

test label: tensor([6745, 3237, 3237, 4377, 4559])
last 10 items from origin dataset: [ 797 4975 6339 6070 4054 6745 3237 3237 4377 4559]


As we expected, in last 10 items, 5 last items are used as label item. So for evaluating this user, we will have 5 sample rows in test data: 
* Input feature: ? -> Output: 6745
* Input feature: ? -> Output: 3237
* Input feature: ? -> Output: 3237
* Input feature: ? -> Output: 4377
* Input feature: ? -> Output: 4559

Now, let check input features in **item_id_list**

In [17]:
input_features['item_id_list']

tensor([[   5,    6,    7,    8,    9,   10, 3566, 2149, 3123, 2739, 3548, 3545,
         3807, 4115,  949, 4476, 4398, 4109, 4449, 4449, 4024, 4570,  377, 5336,
         3491, 5340, 3608, 3608, 5138, 5138, 1880, 5234, 2442, 6000, 6000,  917,
         6092, 5555, 5555, 6257, 3216,  797, 4975, 6339, 6070, 4054,    0,    0,
            0,    0],
        [   5,    6,    7,    8,    9,   10, 3566, 2149, 3123, 2739, 3548, 3545,
         3807, 4115,  949, 4476, 4398, 4109, 4449, 4449, 4024, 4570,  377, 5336,
         3491, 5340, 3608, 3608, 5138, 5138, 1880, 5234, 2442, 6000, 6000,  917,
         6092, 5555, 5555, 6257, 3216,  797, 4975, 6339, 6070, 4054, 6745,    0,
            0,    0],
        [   5,    6,    7,    8,    9,   10, 3566, 2149, 3123, 2739, 3548, 3545,
         3807, 4115,  949, 4476, 4398, 4109, 4449, 4449, 4024, 4570,  377, 5336,
         3491, 5340, 3608, 3608, 5138, 5138, 1880, 5234, 2442, 6000, 6000,  917,
         6092, 5555, 5555, 6257, 3216,  797, 4975, 6339, 6070, 40

We can see:
* For 1st row, it uses all items in training as input features.
* For 2nd row, it uses all items in training + first label as input features
* For 3rd row, it uses all items in training + first label + second label as input features
* ...
* For last row, it uses all items except last item as input features.

In my previous notebooks([here](https://www.kaggle.com/code/astrung/lstm-sequential-modelwith-item-features-tutorial) and [here](https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial/notebook)), **I use last row result for recommendation, so we are missing information from last item. **

So now let fix it- find a new way for using all items

# 2. Custom code for using all items in recommendation

We have seen that last row is missing only last item, so fixxing ideal is simple now:
* copy last row, add last item into it as a new interation(a row in test dataset)
* make prediction with new interation

So now let train a dummy model for testing it

### 1. Make dummy model

In [18]:
# model loading and initialization
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

GRU4Rec(
  (item_embedding): Embedding(7330, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 551104
epoch 0 training [time: 14.43s, train loss: 3384.6924]
Saving current: saved/GRU4Rec-Mar-20-2022_03-26-16.pth
epoch 1 training [time: 10.94s, train loss: 3260.1609]
Saving current: saved/GRU4Rec-Mar-20-2022_03-26-16.pth


In [19]:
model.eval()

GRU4Rec(
  (item_embedding): Embedding(7330, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)

In [20]:
input_features['item_id_list'].shape

torch.Size([5, 50])

Our sequence items is always have fix length(50). So if we have more than 50 items, we need to drop earlier items, and if there are less than 50 items, we need to add a padding(0) into input item features. As example:
* If last row input = [3, 4, 7,..., 20, 0, 0 ,0] (47 items < 50 item, so we have padding), after adding id=30, we will have input = [3, 4, 7,..., 20, 30, 0 ,0] Now our sequence lenght = 48 items.
* If last row input = [3, 4, 7,..., 20, 9, 10 ,12] (50 items), after adding id=30, we will have input = [4, 7,..., 9, 10, 12, 30] (drop first item and add last item).Now our sequence lenght still = 50 items.

Now let implement it.

First let extract last row from all interation when internal_user_id = 2 

In [21]:
index = np.isin(dataset[dataset.uid_field].numpy(), [2])
input_interaction = dataset[index]
input_interaction

The batch_size of interaction: 50
    user_id, torch.Size([50]), cpu, torch.int64
    item_id, torch.Size([50]), cpu, torch.int64
    timestamp, torch.Size([50]), cpu, torch.float32
    item_length, torch.Size([50]), cpu, torch.int64
    item_id_list, torch.Size([50, 50]), cpu, torch.int64
    timestamp_list, torch.Size([50, 50]), cpu, torch.float32


Now let add last item into sequences, and make new interaction.
We also need to edit sequence lenght (without padding)

In [22]:
import torch
from recbole.data.interaction import Interaction

def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[input_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))

test = {
            'item_id_list': add_last_item(input_interaction, input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
new_inter = Interaction(test)
new_inter

The batch_size of interaction: 1
    item_id_list, torch.Size([1, 50]), cpu, torch.int64
    item_length, torch.Size([1]), cpu, torch.int64


Interaction for GRU4Rec model need to have only `item_id_list` and `item_lenght`. You can drop other key.
If you want more information, you can check [GRU4Rec code](https://recbole.io/docs/_modules/recbole/model/sequential_recommender/gru4rec.html#GRU4Rec)

Then we can apply the remaining prediction code from [full_sort_scores](https://recbole.io/docs/_modules/recbole/utils/case_study.html#full_sort_scores)


In [23]:
new_inter = new_inter.to(config['device'])
new_scores = model.full_sort_predict(new_inter)
new_scores = new_scores.view(-1, test_data.dataset.item_num)
new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf

So now by combining all fragments,we have a new function for predicting with all item in dataset. You can use this custom code for all sequential model

In [24]:
import torch
from recbole.data.interaction import Interaction

def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[old_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))

def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index]
        test = {
            'item_id_list': add_last_item(input_interaction, 
                                          input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 10)

In [25]:
predict_for_all_item('0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef', 
                     dataset, model) # we feed directly origin dataset, not train data or test data

torch.return_types.topk(
values=tensor([[5.4805, 5.2197, 5.1685, 5.1523, 5.0042, 4.9986, 4.9978, 4.9962, 4.9898,
         4.9557]], device='cuda:0'),
indices=tensor([[5412,  782, 2668, 1608, 4579, 2426,  294, 2427, 5835, 2669]],
       device='cuda:0'))

Congratulation !!!.Now you can use all data as train set, don't need for a test set, but still can predict directly from dataset without testset.Now let apply it into our previous notebook.

I have create new notebooks for applying our customize function for using all items as input for recommendation:
* Using only interactions: https://www.kaggle.com/astrung/sequential-model-fixed-missing-last-item
* Using interactions with item features: https://www.kaggle.com/code/astrung/lstm-model-with-item-infor-fix-missing-last-item

Please check and upvote it